<a href="https://colab.research.google.com/github/lakhanrajpatlolla/aiml-learning/blob/master/Cohort24RAGSession.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai faiss-cpu tiktoken python-dotenv pypdf langchain-community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 433.9/433.9 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.52
    Uninstalling langchain-core-0.3.52:
      Successfully uninstalled langchain-core-0.3.52


In [ ]:
!pip show langchain

Name: langchain
Version: 0.3.23
Summary: Building applications with LLMs through composability
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: langchain-core, langchain-text-splitters, langsmith, pydantic, PyYAML, requests, SQLAlchemy
Required-by: langchain-community


In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
OPEN_AI_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
pdf_path = "/content/doc202517482201.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load()
print(f"🤷‍♂️loaded {len(pages)} Pages")
splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
chunks = splitter.split_documents(pages)
print(f"🤷‍♂️split into {len(chunks)} chunks")

🤷‍♂️loaded 10 Pages
🤷‍♂️split into 47 chunks


In [ ]:
embeddings = OpenAIEmbeddings(openai_api_key=OPEN_AI_KEY)
vectorstore = FAISS.from_documents(chunks, embeddings)
print("Vector store created")

Vector store created


In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI()
query = "what is GDP growth rate of india for 2024-25 ?"
direct_response=client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": query}
    ]
)
print("Gdp without RAG")
print(direct_response.choices[0].message.content.strip())

Gdp without RAG
It is not possible to accurately predict the GDP growth rate for India in 2024-25 as it depends on various economic factors and events that may influence the economy in the future. Projections and estimates for future GDP growth rates are typically made closer to the actual time period based on current economic conditions and trends.


In [ ]:
results = vectorstore.similarity_search(query,k=3)
context = "\n\n".join([doc.page_content for doc in results])

In [ ]:
#Build Augmentation
prompt = f"""
use the following document context to answer the question.
context:
{context}
Question:{query}
Answer:"""
rag_response = client.chat.completions.create(
   model = "gpt-3.5-turbo",
   messages=[
       {"role": "system", "content": "You answer based only on the given document context."},
       {"role": "user", "content": prompt}
   ]

)
print("GPT with RAG Answer")
print(rag_response.choices[0].message.content.strip())

GPT with RAG Answer
The GDP growth rate of India for 2024-25 is estimated at 6.4%.
